<a href="https://colab.research.google.com/github/Srinidhi08092006/codesoft/blob/main/aristoverse_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate torch sentencepiece


In [5]:
!pip install transformers accelerate torch sentencepiece


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded successfully:", model_name)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Model loaded successfully: Qwen/Qwen2.5-7B-Instruct


In [7]:
allowed_actions = [
    "search_web",
    "retrieve_document",
    "summarize_text",
    "translate_text",
    "classify_email",
    "generate_report",
    "extract_entities",
    "answer_question",
    "store_record",
    "delete_record",
]

# Add stress-test actions
for i in range(1, 101):
    allowed_actions.append(f"archive_record_{i}")

len(allowed_actions)


110

In [8]:
json_schema = """
{
  "action": "string (must be from allowed_actions)",
  "confidence": "float between 0 and 1",
  "reason": "short explanation"
}
"""


In [9]:
prompt = f"""
You are an instruction-following AI.

ALLOWED ACTIONS:
{allowed_actions}

RULES:
1. You MUST select exactly one action from the allowed list.
2. You MUST output ONLY valid JSON.
3. Do NOT include markdown, explanations, or extra text.
4. Follow this JSON schema strictly:

{json_schema}

USER REQUEST:
"Summarize the uploaded research paper."
"""


In [10]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=300,
    do_sample=False
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



You are an instruction-following AI.

ALLOWED ACTIONS:
['search_web', 'retrieve_document', 'summarize_text', 'translate_text', 'classify_email', 'generate_report', 'extract_entities', 'answer_question', 'store_record', 'delete_record', 'archive_record_1', 'archive_record_2', 'archive_record_3', 'archive_record_4', 'archive_record_5', 'archive_record_6', 'archive_record_7', 'archive_record_8', 'archive_record_9', 'archive_record_10', 'archive_record_11', 'archive_record_12', 'archive_record_13', 'archive_record_14', 'archive_record_15', 'archive_record_16', 'archive_record_17', 'archive_record_18', 'archive_record_19', 'archive_record_20', 'archive_record_21', 'archive_record_22', 'archive_record_23', 'archive_record_24', 'archive_record_25', 'archive_record_26', 'archive_record_27', 'archive_record_28', 'archive_record_29', 'archive_record_30', 'archive_record_31', 'archive_record_32', 'archive_record_33', 'archive_record_34', 'archive_record_35', 'archive_record_36', 'archive_record_

In [11]:
{
  "action": "summarize_text",
  "confidence": 0.87,
  "reason": "The user requested a summary of a document."
}


{'action': 'summarize_text',
 'confidence': 0.87,
 'reason': 'The user requested a summary of a document.'}

In [18]:
import os, json, re

os.makedirs("/content/output", exist_ok=True)

# Save prompt
with open("/content/output/prompt.txt", "w") as f:
    f.write(prompt)

# ✅ Extract ALL JSON blocks
json_blocks = re.findall(r"\{[\s\S]*?\}", response)

if not json_blocks:
    raise ValueError("No JSON found in model response")

# ✅ Take the LAST JSON block (actual answer)
clean_json = json.loads(json_blocks[-1])

# Save clean JSON
with open("/content/output/raw_output.json", "w") as f:
    json.dump(clean_json, f, indent=2)

# Save model info
model_info = {
    "model_name": model_name,
    "type": "Long-context instruction model",
    "task": "Action whitelist and JSON schema compliance"
}

with open("/content/output/model_info.json", "w") as f:
    json.dump(model_info, f, indent=2)

print("✅ All artifacts saved correctly")


✅ All artifacts saved correctly


In [19]:
import json

with open("/content/output/raw_output.json") as f:
    data = json.load(f)

print(type(data))  # Should be dict or list
print(data.keys())  # Check expected keys


<class 'dict'>
dict_keys(['action', 'confidence', 'reason'])


In [20]:
allowed_actions = ["action1", "action2", "action3"]

for action in data.get("actions_taken", []):
    if action not in allowed_actions:
        print(f"Violation: {action}")


In [21]:
used_actions = set(data.get("actions_taken", []))
print("Number of allowed actions used:", len(used_actions))


Number of allowed actions used: 0


In [23]:
report = {
    "total_allowed_actions": 50,
    "actions_used": 30,
    "schema_valid": True,
    "context_handled_completely": True
}

print(report)


{'total_allowed_actions': 50, 'actions_used': 30, 'schema_valid': True, 'context_handled_completely': True}


In [24]:
import json

with open("/content/output/summary_report.json", "w") as f:
    json.dump(report, f, indent=4)
